HERE NOW


HIGH QUALITY

In [2]:
# Install necessary libraries if not installed
# !pip install diffusers[training] datasets accelerate torch torchvision matplotlib

import os
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from dataclasses import dataclass
from datasets import load_dataset
from diffusers import UNet2DModel, DDPMScheduler, DDPMPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from tqdm.auto import tqdm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define project directory
project_dir = "/content/drive/MyDrive/LeafImageGeneration"
dataset_dir = os.path.join(project_dir, "dataset")
results_dir = os.path.join(project_dir, "results")

# Create directories if not exist
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

print(f"✅ Directories set up: {dataset_dir}, {results_dir}")


ModuleNotFoundError: No module named 'datasets'

In [11]:
# Define Training Configuration
@dataclass
class TrainingConfig:
    image_size = 128  # Increased image size for high-quality images (previously 64)
    train_batch_size = 8  # Training batch size
    eval_batch_size = 10  # Generate 10 images per evaluation
    num_epochs = 50  # Total training epochs
    gradient_accumulation_steps = 2  # Helps with smaller batches
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 50  # Save images after the whole training
    save_model_epochs = 15  # Save model every 15 epochs
    mixed_precision = "fp16"  # Save memory
    output_dir = results_dir  # Save generated images in results folder
    dataset_name = "imagefolder"
    push_to_hub = False
    seed = 0

config = TrainingConfig()

# Load the dataset from Google Drive
dataset = load_dataset("imagefolder", data_dir=dataset_dir, split="train")

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)

# DataLoader
train_dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=config.train_batch_size, shuffle=True, num_workers=2
)


Resolving data files:   0%|          | 0/1478 [00:00<?, ?it/s]

In [12]:
# Define Model
model = UNet2DModel(
    sample_size=config.image_size,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 256, 256),
    down_block_types=("DownBlock2D", "DownBlock2D", "AttnDownBlock2D", "DownBlock2D"),
    up_block_types=("UpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D")
)

# Define Noise Scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=500)

# Optimizer & Learning Rate Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=config.lr_warmup_steps, num_training_steps=(len(train_dataloader) * config.num_epochs)
)


In [13]:
# Training Loop
def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    accelerator = Accelerator(mixed_precision=config.mixed_precision, gradient_accumulation_steps=config.gradient_accumulation_steps)
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(model, optimizer, train_dataloader, lr_scheduler)

    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (clean_images.shape[0],), device=clean_images.device)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                noise_pred = model(noisy_images, timesteps).sample
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            progress_bar.set_postfix(loss=loss.item())

        # Save model periodically
        if epoch % config.save_model_epochs == 0:
            accelerator.unwrap_model(model).save_pretrained(config.output_dir)
            print(f"💾 Model saved at epoch {epoch}")

    # After all epochs, generate high-quality images
    pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)
    generate_images(config, pipeline)

# Run Training
train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)


  0%|          | 0/185 [00:00<?, ?it/s]

💾 Model saved at epoch 0


  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

...

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

NameError: name 'generate_images' is not defined

In [21]:
# Function to save the model
def save_model(model, output_dir):
    # Save the model weights
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    print(f"💾 Model saved at {output_dir}")

# After training loop (or any point you wish to save the model)
save_model(model, config.output_dir)


💾 Model saved at /content/drive/MyDrive/LeafImageGeneration/results
